<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/Pattern_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial Setup

In [1]:
#Importing Libraries
#https://medium.datadriveninvestor.com/automating-technical-analysis-pattern-recognition-bcbc85e4ec3c
import pandas as pd
import datetime
from datetime import date
from datetime import timedelta
import matplotlib.pyplot as plt
import yahoo_fin.stock_info as si
from pytictoc import TicToc
import talib
import numpy as np
import mplfinance as mpf
from talib import abstract
t=TicToc()
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: ignored

### Functions

In [ ]:
##Getting Patterns

def get_patterns(data=None,ticker_column = 'ticker',mute=False):
    #from talib import abstract
    result = pd.DataFrame()
    tickers = data[ticker_column].unique()
    x=[]
    attr = talib.get_function_groups()['Pattern Recognition']
    for i in tickers:
        df = data[data[ticker_column]==i]
        for a in attr:
            df[a] = (getattr(abstract,a)(df))/100
        result = pd.concat([result,df])
        x.append(i)
        if mute is False:
            print(i + " Patterns Appended Successfully")
            print(result.shape)
            print(str(len(x)) + " of "+ str(len(tickers)) + " Done ")
        else:
            pass
    return result

#Creating Indicators

def get_indicators(data=None,ticker_column = 'ticker',price_column="close",mute=False):
    result = pd.DataFrame()
    tickers = data[ticker_column].unique()
    x=[]
    for i in tickers:
        df = data[data[ticker_column]==i]
        df["Vol_over_30DAvg"] = df["volume"]/df["volume"].rolling(window=30).mean()
        df["Vol_over_10DAvg"] = df["volume"]/df["volume"].rolling(window=10).mean()
        df["Perc_From_200D_H"] = ((df[price_column]/df[price_column].rolling(window=200).max())-1)*100
        df["Perc_From_200D_L"] = ((df[price_column]/df[price_column].rolling(window=200).min())-1)*100
        df["Retracement_200D"] = df[price_column]/(df[price_column].rolling(window=200).max()-df[price_column].rolling(window=30).min())
        df["Perc_From_30D_H"] = ((df[price_column]/df[price_column].rolling(window=30).max())-1)*100
        df["Perc_From_30D_L"] = ((df[price_column]/df[price_column].rolling(window=30).min())-1)*100
        df["Retracement_30D"] = df[price_column]/(df[price_column].rolling(window=30).max()-df[price_column].rolling(window=30).min())
        df["30D/200D_High"] = df[price_column].rolling(window=30).max()/df[price_column].rolling(window=200).max()
        df["30D/200D_Low"] = df[price_column].rolling(window=30).min()/df[price_column].rolling(window=200).min()
        df["Perc_from 60DEMAVG"] = ((df[price_column]/df[price_column].rolling(window=60).mean())-1)*100
        df["Perc_from 30DEMAVG"] = ((df[price_column]/df[price_column].rolling(window=30).mean())-1)*100
        df["Perc_from 200DEMAVG"] = ((df[price_column]/df[price_column].rolling(window=200).mean())-1)*100
        df["Variand_30D/200D"] = df[price_column].rolling(window=30).var()/df[price_column].rolling(window=200).var()
        df = df.dropna()
        df = df.sort_index(ascending=True)
        result = pd.concat([result,df])
        
        if mute is False:
            print(i + " Indicators Appended Successfully")
            print(result.shape)
            x.append(i)
            print(str(len(x)) + " of "+ str(len(tickers)) + " Done ")
        else:
            pass

    return result

## Getting Labels

def get_labels(data = None,ticker_column="ticker", fwd_window=1,price_column="close",threshold_perc=1,mute=False):
    result = pd.DataFrame()
    tickers = data[ticker_column].unique()
    x=[]
    for i in tickers:
        df = data[data[ticker_column]==i]
        df.sort_index(ascending=False)
        df["Change"] = df[price_column].pct_change(periods=fwd_window).shift(-fwd_window)
        df["Change"]=np.where(abs(df["Change"])>=(threshold_perc/100),df["Change"]/abs(df["Change"]),0)
        result= pd.concat([result,df],axis=0)
        if mute is False:    
            print(i + " Labels Appended Successfully")
            print(result.shape)
            x.append(i)
            print(str(len(x)) + " of "+ str(len(tickers)) + " Done ")
        else:
            pass
    result.dropna(inplace=True)
    return result


## Compile Data

def compile_data(data=None,ticker_column = 'ticker',price_column="close",fwd_window=1,threshold_perc=1,mute=False,train=True):
    print("Compiling pattern Signals")
    data_patterns = get_patterns(data=data,ticker_column=ticker_column,mute=mute)
    print ("---- Pattern Signals Attached")
    print("Compiling Indicators")
    data_indicators = get_indicators(data=data_patterns,ticker_column=ticker_column,price_column=price_column,mute=mute)
    print ("---- Indicators Attached")
    print ("Creating Labels")
    data_labelled = get_labels(data=data_indicators,ticker_column=ticker_column,price_column="close", fwd_window=fwd_window,threshold_perc=1,mute=mute)
    print("---- All Labels Attached")
    if train is True:
        return data_labelled
    else:
        return data_indicators

#Getting Pattern based signals

def get_pattern_signals(tickers=None,top=5):
    d=pd.DataFrame()
    start_date = date.today() - timedelta(days=200)
    start_date= start_date.strftime('%m/%d/%Y')
    end_date = date.today().strftime('%m/%d/%Y')
    for i in tickers:
        
        a = si.get_data(ticker = i, start_date=start_date,end_date=end_date)
        z = get_patterns(data=a,mute=True).iloc[-1,6:]
        
        d= pd.concat((d,z),axis=1)
        print(i + "   ",end="\r")
    d.columns = d.iloc[0]
    d=d.drop("ticker")
    d=d.sum()/d.count()
    d=d.sort_values(ascending=False)
    top_buy = d[d>0].head(top).sort_values(ascending=False)
    top_sell = d[d<0].tail(top).sort_values(ascending=True)
    print("Top Buy Ideas")
    print(top_buy)
    print("Top Sell Ideas")
    print(top_sell)
    return top_buy,top_sell

## Getting Top MCap Tickers

def get_top_tickers (index_name = 'sp500',top=50, tickers = None):
    ''' For index_name, you can choose from the following:\n
        'sp500'   - S&P 500 Index \n
        'dow'     - Dow Jones Industrial Average \n
        'nasdaq'  - Companies listed on Nasdaq\n
        'nifty50' - For NIFTY50\n
        'ftse100' - For FTSE100\n
        'ftse250' - For FTSE250\n
        Or for any other index not listed above, a list of all the tickers can be passed to filter out Top N companies by Market Capitalization '''
    mcap=[]
    if tickers is None:
        t = "tickers_"+index_name
        tickers = getattr(si,t)()
    else:
        pass
    for i in tickers:
        a= si.get_quote_data(i)
        b=round(a['sharesOutstanding']*a['regularMarketPrice']/1000000)
        mcap.append(b)
    mcap_df = pd.DataFrame()
    mcap_df["Tickers"] = tickers
    mcap_df["MCap_mn"] = mcap
    mcap_df=mcap_df.sort_values(by=["MCap_mn"],ascending=False)
    top_mcap_tickers = mcap_df.head(top)["Tickers"]
    return top_mcap_tickers.to_list()

## Getting Historical Bulk Data for Download

def get_hist_data (tickers=None,time_window =1000,start_date=None,delta =1):
    i_list=[]
    data = pd.DataFrame()
    if start_date is None:
        start_date = date.today() - timedelta(days= time_window)
        start_date= start_date.strftime('%m/%d/%Y')
    else:
        start_date= start_date.strftime('%m/%d/%Y')
    end_date = date.today() - timedelta(days=delta)
    end_date= end_date.strftime('%m/%d/%Y')
    for i in tickers:
        a = si.get_data(ticker = i, start_date=start_date,end_date=end_date)
        data = pd.concat([data,a],axis=0)
        print(i+" Appended Successfully")
        print(data.shape)
        i_list.append(i)
        print(str(len(i_list)) + " of "+ str(len(tickers)) + " Done ")
        data.drop_duplicates(inplace = True)
        data.dropna(inplace=True)
    return data

#### Selecting Top MCap Companies

In [ ]:
top_100 = get_top_tickers(index_name='sp500',top=100)
top_100

In [ ]:
top_100[:10]

## Simple Pattern Based Signal Generation

In [ ]:
patterns = talib.get_function_groups()['Pattern Recognition']
print("No. of patterns - "+ str(len(patterns)))
print(patterns[0:5])

In [ ]:
aapl = get_hist_data(tickers=["aapl"],time_window=1000)
aapl.tail()

In [ ]:
aapl_patterns = get_patterns(data=aapl)
aapl_patterns.iloc[:,-4:].tail()

In [ ]:
aapl_patterns.iloc[:,:5].head()

In [ ]:
aapl_patterns["sum_flags"] = aapl_patterns[patterns].sum(axis=1)
print("Total Observation - " + str(aapl_patterns.shape[0]))
print("Observations with Flag - " + str(aapl_patterns[aapl_patterns["sum_flags"]!=0].shape[0]) )

In [ ]:
print(patterns)

In [ ]:
# Visualizing a pattern on candle stick chart

aapl_engulfing = aapl_patterns['CDLENGULFING']
aapl_engulfing_bullish = (aapl_engulfing.replace([-1,0],np.nan))*aapl_patterns.low*0.99
aapl_engulfing_bearish = (-aapl_engulfing.replace([1,0],np.nan))*aapl_patterns.high*1.01
%matplotlib ipympl
addplot = [mpf.make_addplot(aapl_engulfing_bullish,type='scatter',color = 'g',marker = '^'), 
            mpf.make_addplot(aapl_engulfing_bearish,type='scatter',color = 'r',marker = 'v')]
mpf.plot(aapl_patterns.iloc[:,:5],addplot=addplot,type='candle',style= 'yahoo')

In [ ]:
top_100[:10]

In [ ]:
# Pattern based signal generation
top_buy,top_sell=get_pattern_signals(tickers=top_100,top=5)